In [101]:
import requests
import pandas as pd
import numpy as np
import sidetable
from geopy.geocoders import Nominatim
import mysql.connector

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import os
from dotenv import load_dotenv
load_dotenv()

True

In [102]:
class Extraccion_limpieza:

    def __init__(self, lista_paises, nombre_fichero_coord):

        self.lista_paises = lista_paises
        self.nombre_fichero_coord = nombre_fichero_coord

    def extraer_df_api(self):

        """Crea una lista con los 3 países que queremos estudiar y realiza la llamada a la API y extracción de datos de la misma para cada uno de los países.
        Después crea un dataframe con los datos de cada país y los unifica en un único dataframe.
        Args:
            No recibe.
        Returns:
            df: dataframe unido con los datos de los 3 países.
        """

        df_unido = pd.DataFrame()

        for pais in self.lista_paises:

            url = f'http://universities.hipolabs.com/search?country={pais}'

            response = requests.get(url=url)

            response.status_code

            response.reason
            
            df = pd.json_normalize(response.json())

            df_unido = pd.concat([df_unido, df], ignore_index = True)
        
        return df_unido
    

    def homogeneizar_columnas(self, df, col = 'domains'):

        """Reemplaza los guiones de los nombres de las columnas del dataframe por barra baja, y
            elimina columnas redundantes.
        Args:
            df: dataframe original.
            col: columna redundante, parámetro por defecto.
        Returns:
            df: dataframe modificado.
        """
        
        df.rename(columns = {col: col.replace('-', '_') for col in df.columns}, inplace = True)
        df.drop(col, axis = 1, inplace = True)

        return df
    

    def aplicar_explode(self, df, col = 'web_pages'):

        """Aplica método explode a columnas con más de un valor por registro.
        Args:
            df: dataframe original.
            col: col a aplicar explode, parámetro por defecto.
        Returns:
            df: dataframe modificado.
        """
        
        return df.explode(col, ignore_index= True)
    

    def eliminar_duplicados(self, df, col = 'name'):

        """Elimina las filas duplicadas en base a una columna que le indiquemos.
        Args:
            df: dataframe.
            col: columna sobre la que buscar duplicados.
        Returns:
            df: dataframe sin las filas duplicadas.
        """

        if df.duplicated([col]).sum() > 0:
            print(f'Tenemos {df.duplicated([col]).sum()} duplicados en la columna "{col}", los eliminaremos.')
            return df.drop_duplicates(subset = col, inplace = True, ignore_index = True)
        
        else:
            print(f'No tenemos duplicados en la columna {col}.')

# preguntar
    def convertir_a_nan(self, valor):

        """convierte los valores None a np.nan.
        Args:
            valor (str): valor None
        Returns:
            valor (np.nan): valor np.nan
        """

        if valor == None:
            return np.nan
        else:
            return valor


    def imputar_nulos(self, df, col):

        """convierte los valores nulos al valor de string 'Unknow'.
        Args:
            df (df): dataframe.
            col (col): columna sobre la que queremos imputar los valores nulos.
        Returns:
            df: dataframe con nulos imputados.
        """

        return df[col].replace(np.nan, 'Unknow', inplace = True)


    def sacar_latitud_longitud_geopy(self, lista):

        """Usando la librería Geopy, obtiene la latitud y la longitud de una lista de provincias que
        le indiquemos, con ello crea un dataframe con las columnas provincia, latitud y longitud.
        Si la provincia es desconocida, devuelve un valor nulo para latitud y longitud.
        Args:
            lista (list): lista con localizaciones.
        Returns:
            df (df): dataframe de 3 columnas (provincia, latitud y longitud).
        """

        lista_latitud = []
        lista_longitud = []

        for estado in lista:

            if estado != 'Unknow':
                geo = Nominatim(user_agent = 'Natalia')
                localizacion = geo.geocode(estado)
                lista_latitud.append(localizacion.latitude)
                lista_longitud.append(localizacion.longitude)
        
            else:
                lista_latitud.append(np.nan)
                lista_longitud.append(np.nan)

        diccionario = {
            'state_province' : lista,
            'latitude' : lista_latitud,
            'longitude' : lista_longitud
            }

        df = pd.DataFrame(diccionario)
        df.to_csv(f'../data/coordenadas_{self.nombre_fichero_coord}.csv') 

        return df


    def abrir_latitud_longitud_fichero(self):

        """Abre el archivo guardado en el anterior método. Este método está pensado para poder trabajar con unos datos
        extraídos previamente de la librería geopy en caso de que en el momento actual la librería no funcione bien.
        Args:
            No recibe.
        Returns:
            df (df): dataframe de 3 columnas guardado.
        """       
    
        return pd.read_csv(f'../data/coordenadas_{self.nombre_fichero_coord}.csv', index_col = 0)


    def mergear_dfs(self, df1, df2, col):

        """Mergea los 2 dataframes creados con esta clase.
        Args:
            df1 (df): primer dataframe a unir.
            df2 (df): segundo dataframe a unir.
            col (df): columna por la que queremos unir los dataframes.
        Returns:
            df (df): dataframe unido.
        """
    
        return df1.merge(df2, on = col)



In [103]:
lista_paises = ['United States', 'Canada' , 'Argentina']

In [104]:
api = Extraccion_limpieza(lista_paises, 'US_Can_Arg')
api

In [105]:
df_universidades = api.extraer_df_api()
df_universidades.head()

,domains,country,name,web_pages,state-province,alpha_two_code
0,[marywood.edu],United States,Marywood University,[http://www.marywood.edu],None,US
1,[lindenwood.edu],United States,Lindenwood University,[http://www.lindenwood.edu/],None,US
2,[sullivan.edu],United States,Sullivan University,[https://sullivan.edu/],None,US
3,[fscj.edu],United States,Florida State College at Jacksonville,[https://www.fscj.edu/],None,US
4,[xavier.edu],United States,Xavier University,[https://www.xavier.edu/],None,US


In [106]:
api.homogeneizar_columnas(df_universidades).head()

,country,name,web_pages,state_province,alpha_two_code
0,United States,Marywood University,[http://www.marywood.edu],None,US
1,United States,Lindenwood University,[http://www.lindenwood.edu/],None,US
2,United States,Sullivan University,[https://sullivan.edu/],None,US
3,United States,Florida State College at Jacksonville,[https://www.fscj.edu/],None,US
4,United States,Xavier University,[https://www.xavier.edu/],None,US


In [107]:
df_universidades_exp = api.aplicar_explode(df_universidades)

In [108]:
df_universidades_exp.head(1)

,country,name,web_pages,state_province,alpha_two_code
0,United States,Marywood University,http://www.marywood.edu,None,US


In [109]:
df_universidades_exp[df_universidades_exp['name'] == 'Cégep de Saint-Jérôme']

,country,name,web_pages,state_province,alpha_two_code
4554,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA
4555,Canada,Cégep de Saint-Jérôme,https://ccmt.cstj.qc.ca,Quebec,CA
4556,Canada,Cégep de Saint-Jérôme,https://ccml.cstj.qc.ca,Quebec,CA
4708,Canada,Cégep de Saint-Jérôme,https://www.cstj.qc.ca,Quebec,CA
4709,Canada,Cégep de Saint-Jérôme,https://ccmt.cstj.qc.ca,Quebec,CA
4710,Canada,Cégep de Saint-Jérôme,https://ccml.cstj.qc.ca,Quebec,CA


In [110]:
api.eliminar_duplicados(df_universidades_exp)

Tenemos 2543 duplicados en la columna "name", los eliminaremos.


In [111]:
df_universidades_exp.stb.missing()

,missing,total,percent
state_province,2204,2491,88.478523
country,0,2491,0.000000
name,0,2491,0.000000
web_pages,0,2491,0.000000
alpha_two_code,0,2491,0.000000


In [112]:
def convertir_a_nan(valor):
    """convierte los valores None a np.nan.

    Args:
        valor (str): valor None

    Returns:
        valor (np.nan): valor np.nan
    """

    if valor == None:
        return np.nan
    else:
        return valor

In [113]:
df_universidades_exp['state_province'] = (df_universidades_exp['state_province'].apply(convertir_a_nan))

In [114]:
df_universidades_exp.stb.missing()

,missing,total,percent
state_province,2204,2491,88.478523
country,0,2491,0.000000
name,0,2491,0.000000
web_pages,0,2491,0.000000
alpha_two_code,0,2491,0.000000


In [115]:
api.imputar_nulos(df_universidades_exp, 'state_province')

In [116]:
df_universidades_exp.stb.missing()

,missing,total,percent
country,0,2491,0.0
name,0,2491,0.0
web_pages,0,2491,0.0
state_province,0,2491,0.0
alpha_two_code,0,2491,0.0


In [117]:
df_universidades_exp['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'NV', 'Iowa', 'VA',
       'TX', 'Indiana', 'Colorado', 'Ohio', 'IN', 'New York', 'CA',
       'Illinois', 'New Hampshire', 'North Carolina', 'South Carolina',
       'Virginia', 'Washington', 'Missouri', 'California', 'NY', 'ND',
       'MI', 'Florida', 'Michigan', 'GA', 'New York, NY', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Santiago Del Estero',
       'Misiones', 'Catamarca', 'Formosa', 'Jujuy', 'La Rioja',
       'La Pampa', 'San Juan', 'San Luis', 'Tucumán'], dtype=object)

In [118]:
dicc_estados = {
    'NV' : 'Nevada',
    'TX' : 'Texas',
    'IN' : 'Indianapolis',
    'CA' : 'California',
    'VA' : 'Virginia',
    'NY' : 'New York',
    'MI' : 'Michigan', 
    'GA' : 'Georgia', 
    'ND' : 'North Dakota', 
    'New York, NY' : 'New York', 
    'Ciudad Autónoma de Buenos Aires' : 'Buenos Aires'}

In [119]:
df_universidades_exp["state_province"] = df_universidades_exp['state_province'].apply(lambda estado: dicc_estados.get(estado, estado))

In [120]:
df_universidades_exp['state_province'].unique()

array(['Unknow', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán'], dtype=object)

In [121]:
lista_estados = df_universidades_exp['state_province'].unique().tolist()

In [122]:
df_geopy = api.sacar_latitud_longitud_geopy(lista_estados)
df_geopy.head()

,state_province,latitude,longitude
0,Unknow,NaN,NaN
1,Pennsylvania,40.969989,-77.727883
2,Texas,31.263890,-98.545612
3,Utah,39.422519,-111.714358
4,Nevada,39.515882,-116.853723


In [124]:
df_unido = api.mergear_dfs(df_universidades_exp, df_geopy, 'state_province')
df_unido.head()

,country,name,web_pages,state_province,alpha_two_code,latitude,longitude
0,United States,Marywood University,http://www.marywood.edu,Unknow,US,NaN,NaN
1,United States,Lindenwood University,http://www.lindenwood.edu/,Unknow,US,NaN,NaN
2,United States,Sullivan University,https://sullivan.edu/,Unknow,US,NaN,NaN
3,United States,Florida State College at Jacksonville,https://www.fscj.edu/,Unknow,US,NaN,NaN
4,United States,Xavier University,https://www.xavier.edu/,Unknow,US,NaN,NaN


In [152]:
class Carga:
    
    def __init__(self, host, bbdd):

        self.host = host
        self.bbdd = bbdd


    def crear_bbdd(self):

        """Usando mysql.connector, crea en MySQL una base de datos.
        Args:
            nombre de la base de datos (str): el nombre que queremos poner a nuestra base de datos.
        Returns:
            No tiene. Crea directamente la base de datos en MySQL.
        """
        
        conexion = mysql.connector.connect(
                        host = self.host,
                        user= os.getenv('user'),
                        password= os.getenv('password'))
        
        mycursor = conexion.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.bbdd};")
            conexion.commit() 

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


    def crear_tabla(self, query):

        """Usando mysql.connector, crea en MySQL una tabla en una base de datos.
        Args:
            nombre de la base de datos (str): la base de datos donde queremos crear nuestra tabla.
            consulta (str): la consulta con la que creamos la tabla.
        Returns:
            No tiene. Crea directamente la tabla en MySQL.
        """

        conexion = mysql.connector.connect(
                                host = self.host,
                                user= os.getenv('user'),
                                password= os.getenv('password'), 
                                database=f"{self.bbdd}")
        
        mycursor = conexion.cursor()
        
        try: 
            mycursor.execute(query)
        
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


    def insertar_datos(self, query):
    
        """Usando mysql.connector, inserta datos en una tabla de una base de datos.
        Args:
            nombre de la base de datos (str): la base de datos donde queremos insertar los datos.
            consulta (str): la consulta con la que insertamos los datos.
        Returns:
            No tiene. Inserta los datos directamente en la tabla en MySQL.
        """

        conexion = mysql.connector.connect(
                                host = self.host,
                                user= os.getenv('user'),
                                password= os.getenv('password'), 
                                database=f"{self.bbdd}")
        
        mycursor = conexion.cursor()
        
        try: 
            mycursor.execute(query)
            conexion.commit() 

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)


    def sacar_id_estado(self, estado, pais):

        """Usando mysql.connector, selecciona el id de un estado de nuestra base de datos.
        Args:
            nombre de la base de datos (str): la base de datos de la que queremos extraer la información.
            estado (str): estado del que queremos sacar el id.
            pais (str): país al que pertenece el estado del que queremos sacar el id.
        Returns:
            id (str): el id del estado.
        """
        conexion = mysql.connector.connect(
                                host = self.host,
                                user= os.getenv('user'),
                                password= os.getenv('password'), 
                                database=f"{self.bbdd}")
        
        mycursor = conexion.cursor()

        query_sacar_id = f"SELECT id_estado FROM paises WHERE nombre_provincia = '{estado}' AND nombre_pais = '{pais}'"
        
        # puede que el id de la ciudad que estamos intentando insertar no este en nuestra BBDD, de modo que usaremos un try except para evitar errores
        try: 
            mycursor.execute(query_sacar_id)
            id_est = mycursor.fetchall()[0][0]
            
            return id_est
        
        except: 
            return "Lo siento, no tenemos ese estado en la BBDD y por lo tanto no te podemos dar su id. "
        
    
    def cerrar_conexion(self):

        """Cierra la conexión con MySQL."""

        self.conexion.close()


In [153]:
carga = Carga('localhost', 'Universidades')

In [154]:
carga.crear_bbdd()

In [155]:
tabla_paises = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`paises` (
    `id_estado` INT NOT NULL AUTO_INCREMENT,
    `nombre_pais` VARCHAR(45) NOT NULL,
    `nombre_provincia` VARCHAR(45),
    `latitud` FLOAT,
    `longitud`FLOAT,
    PRIMARY KEY (`id_estado`));
'''

tabla_universidades = '''
CREATE TABLE IF NOT EXISTS `Universidades`.`universidades` (
    `id_universidades` INT NOT NULL AUTO_INCREMENT,
    `nombre_universidad` VARCHAR(100) NOT NULL,
    `pagina_web` VARCHAR(100),
    `paises_id_estado` INT NOT NULL,
    PRIMARY KEY (`id_universidades`),
    CONSTRAINT `fk_universidades_paises`
        FOREIGN KEY (`paises_id_estado`)
        REFERENCES `Universidades`.`paises` (`id_estado`));
ENGINE = InnoDB;
'''

In [156]:
carga.crear_tabla(tabla_paises)

In [157]:
carga.crear_tabla(tabla_universidades)

In [159]:
for indice, fila in df_unido.iterrows():

        latitud = fila['latitude']
        longitud = fila['longitude']

        if pd.isna(latitud):
                latitud = 'NULL'

        if pd.isna(longitud):
                longitud = 'NULL'

        id_estado = carga.sacar_id_estado(fila['state_province'],fila['country'])

        if type(id_estado) is not int:

                insertar_paises = f"""
                        INSERT INTO `Universidades`.`paises` (nombre_pais, nombre_provincia, latitud, longitud) 
                        VALUES ( "{fila['country']}", "{fila['state_province']}", {latitud}, {longitud});
                        """
                carga.insertar_datos(insertar_paises)
        else:
                pass

In [160]:
for indice, fila in df_unido.iterrows():

        id_estado = carga.sacar_id_estado(fila['state_province'], fila['country'])
        insertar_universidades = f"""
                INSERT INTO `Universidades`.`universidades` (nombre_universidad, pagina_web, paises_id_estado) 
                VALUES ( "{fila['name']}", "{fila['web_pages']}", {id_estado});
                """
        
        carga.insertar_datos(insertar_universidades)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza"", "http://www.umaza.edu.ar/", 3)' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza"", "http://www.umaza.edu.ar/", 3)' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'San Juan Bosco"", "http://www.unp.edu.ar/", 3)' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'San Juan Bosco"", "http://www.unp.edu.ar/", 3)' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server ver